In [3]:
import random
import time
#--------------------------------------------------------------------------------------------------------------#
class Dice:
    def __init__(self):
        self.die1 = 0
        self.die2 = 0
        self.total = 0
#--------------------------------------------------------------------------------------------------------------#       
    def rollDice(self):
        self.die1 = random.randint(1,6)
        self.die2 = random.randint(1,6)
        self.total = self.die1 + self.die2
        
        return self.die1, self.die2, self.total
#----------------------------------------------------------------------------------------------------------------------#
class Table(Dice):
    def __init__(self):
        super().__init__()
        self.Point = False #the point is initialized to False since a point has not yet been set
#----------------------------------------------------------------------------------------------------------------------#
class Player(Table):
    def __init__(self):
        super().__init__()
        self.name = input("Please enter the player name: ") #get the player's name
        self.bankroll = 0
        
        #try-except statement to get bankroll from the player
        while True:
            try:
                self.bankroll = int(input("Please enter your bankroll (buy in): $"))
                assert(self.bankroll>0)
                break
            except:
                print("Please enter a valid bankroll")
        
#----------------------------------------------------------------------------------------------------------------------#
class Bets(Player):
    
    def __init__(self):
        super().__init__()
        self.bet = 0
        self.passBet = 0
        self.doNotPassBet = 0
        self.oddsBet = 0
        self.oddsWinnings = 0
        self.oddsBetMax = 5*self.bet
        self.point = 0
        self.gameOn = True
        self.bet_loss_type = ""
        self.bet_loss_amount = 0
#--------------------------------------------------------------------------------------------------------------#                
    def insufficent_funds(self):
        while True:
            if self.bet > self.bankroll:
                try: 
                    self.bet = int(input("Insufficient funds: Player " + self.name + " has a bankroll of $"+str(self.bankroll)+". Please enter a new bet: $"))
                    assert(self.bet <= self.bankroll)
                    assert(self.bet > 0)
                    break
                except:
                    print("Insufficient funds or invalid bet: Player " + self.name + " has a bankroll of $"+str(self.bankroll)+". Please increase bankroll to play")
            else:
                break
#--------------------------------------------------------------------------------------------------------------#    
    def printMainMenu(self):
        print("------------------MAIN MENU--------------------------")
        print("1) Pass Line Bet")
        print("2) Do Not Pass Line Bet")
        print("3) Increase buy in")
        print("4) Print bankroll")
        print("5) Cash out & Walk Away")
#--------------------------------------------------------------------------------------------------------------#
    
    def PassLineBet(self):
        self.passBet = self.ingestBet()
        self.shooter()

        if self.total == 2 or self.total == 3 or self.total == 12:
            
            self.bet_loss("pass line bet", self.passBet)
            self.Point = False
            
        elif self.total == 7 or self.total == 11:
            self.bet_won("pass line bet", self.passBet)
            self.Point = False
        else:
            time.sleep(1)
            print("POINT IS SET TO " + str(self.total))
            self.point = self.total
            self.Point = True

        while self.Point:
            
            if (self.bankroll - self.passBet) == 0:
                print("Odds is not available since bankroll is $0")
            else:
                self.Odds()
            
            if self.total == 7:
                
                self.bet_loss("pass line bet", self.passBet)
                time.sleep(1)
                self.Point = False
                
            elif self.total == self.point:
                self.bet_won("pass line bet", self.passBet)
                self.Point = False
        self.Reconcile()
#--------------------------------------------------------------------------------------------------------------#
    def DoNotPassLineBet(self):
        
        self.doNotPassBet = self.ingestBet()
        self.shooter()
        if self.total == 7 or self.total == 11:
            self.bet_loss("do not pass line bet", self.doNotPassBet)
            self.Point = False
        elif self.total == 2 or self.total == 3 or self.total == 12:
            self.bankroll += self.doNotPassBet
            self.bet_won("do not pass line bet", self.doNotPassBet)
            self.Point = False
        else:
            print("POINT IS SET TO " + str(self.total))
            self.point = self.total
            self.Point = True

        while self.Point:
            self.shooter()

            if self.total == 7:
                self.bet_won("do not pass line bet", self.doNotPassBet)
                self.Point = False
            elif self.total == self.point:
                self.bet_loss("do not pass line bet", self.doNotPassBet)
                self.Point = False
        self.Reconcile()
#--------------------------------------------------------------------------------------------------------------#  
    def bettingTurn(self):
        
        choice = int(input())
        if choice == 1:
            if self.bankroll <= 0:
                print("Please increase bankroll before playing")
                bets.printMainMenu()
                bets.bettingTurn()
            else:
                bets.PassLineBet()
        elif choice == 2:
            if self.bankroll <= 0:
                print("Please increase bankroll before playing")
                bets.printMainMenu()
                bets.bettingTurn()
            else:
                bets.DoNotPassLineBet()
        elif choice == 3:
            self.bankroll += int(input("Enter new buy in: $"))
            print("Your bankroll is now: $"+str(self.bankroll))
            bets.printMainMenu()
            bets.bettingTurn()
        elif choice == 4:
            print("Your bankroll is: $"+str(self.bankroll))
            bets.printMainMenu()
            bets.bettingTurn()
        elif choice == 5:
            bets.gameOn = False
            print("Thanks for playing! Your bank is: $"+str(self.bankroll))
#--------------------------------------------------------------------------------------------------------------#
    def ingestBet(self):
        self.bet = int(input("Enter Bet: $"))
        self.insufficent_funds()
        return self.bet
#--------------------------------------------------------------------------------------------------------------#
    def bet_made(self):
        if self.passBet > 0:
            print("Pass Line Bet: $"+str(self.passBet))
        if self.doNotPassBet > 0:
            print("Don't Pass Line Bet: $"+str(self.doNotPassBet))
            
    def bet_loss(self, betType, betAmount):
        self.bankroll -= betAmount
        print("--------------------------------------")
        print("| You loss: "+betType+" | "+"Amount: $"+str(betAmount) + " |")
        print("--------------------------------------")
        
    def bet_won(self, betType, betAmount):
        self.bankroll += betAmount
        print("------------------------------------")
        print("| You Win: " + betType + " | "+"Amount: $"+str(betAmount) + " |")
        print("------------------------------------")

#--------------------------------------------------------------------------------------------------------------#
    def shooter(self):
        if self.passBet > 0 or self.doNotPassBet > 0:
            time.sleep(1)
            D1, D2, total = Dice.rollDice(self)
            print("Rolling dice... " + str(D1) + " and " +str(D2))
            
    def Odds(self):
        playOdds = False
        valid = False
        
        while playOdds == False:
            choice = (input("Would you like to play odds? (y/n)"))
            
            if choice == 'y':
                
                while valid == False:
                    
                    self.oddsBet = int(input("Enter total odds bet (Max is 5x): $"))
                   
                    if self.oddsBet > (5*self.passBet):
                        print("You've reached the max bet. Please decrease bet below 5x")
                    elif self.oddsBet > self.bankroll:
                        print("You've exceeded your bankroll")
                    else:
                        valid = True
            
                playOdds = True
                self.shooter()
                if self.total == self.point == 4 or self.total == self.point == 10:
                    self.bet_won("odds bet", self.oddsBet*2)
                elif self.total == self.point == 5 or self.total == self.point == 9:
                    self.bet_won("odds bet", self.oddsBet*(3/2))
                elif self.total == self.point == 6 or self.total == self.point == 8:
                    self.bet_won("odds bet", self.oddsBet*(6/5))
                elif self.total == 7:
                    self.bet_loss("odds bet", self.oddsBet)
                
            elif choice == 'n':
                print("")
                playOdds = True
                
            else:
                print("Please enter 'y' for yes or 'n' for no")
        
        
#--------------------------------------------------------------------------------------------------------------#
    def Reconcile(self):
        print("----Reconcile Bets------")
        print("Your bankroll is now: "+str(self.bankroll))
        
#--------------------------------------------------------------------------------------------------------------#

#Print the main menu
print("------------------------------------------")
print("      Welcome to the game of craps")
print("------------------------------------------")
print("Starting game ...")
time.sleep(1)

#instantiate Bets class as bets object
bets = Bets()

#continue playing the game until the player selects 5) Cash Out
while bets.gameOn:
    bets.printMainMenu() #prints the main menu with 5 choices
    bets.bettingTurn() #initiates choice selection




    




------------------------------------------
      Welcome to the game of craps
------------------------------------------
Starting game ...
Please enter the player name: j
Please enter your bankroll (buy in): $10
------------------MAIN MENU--------------------------
1) Pass Line Bet
2) Do Not Pass Line Bet
3) Increase buy in
4) Print bankroll
5) Cash out & Walk Away
1
Enter Bet: $10
Rolling dice... 2 and 6
POINT IS SET TO 8
Odds is not available since bankroll is $0
------------------------------------
| You Win: pass line bet | Amount: $10 |
------------------------------------
----Reconcile Bets------
Your bankroll is now: 20
------------------MAIN MENU--------------------------
1) Pass Line Bet
2) Do Not Pass Line Bet
3) Increase buy in
4) Print bankroll
5) Cash out & Walk Away


KeyboardInterrupt: Interrupted by user